<h4> Question: Use any recommendation dataset from Kaggle and perform content based and collobarative filtering on the dataset. </h4>

Inspiration: https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

There's a lot of preprocessing & cool steps in this article, do check them out!

For Theory: https://towardsdatascience.com/a-complete-guide-to-recommender-system-tutorial-with-sklearn-surprise-keras-recommender-5e52e8ceace1

<h3> Content-Based Recommendation System </h3>

In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
data = pd.read_csv("Netflix_Dataset_Movie.csv")

data.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [3]:
def WordCleaner(word):
    word = word.lower()
    letters = list(word)
    
    word = ""
    
    for letter in letters:
        if letter.isalpha() or letter.isnumeric():
            word += letter
    
    return word

In [4]:
def SentenceCleaner(sentence):
    cleaned_sentence = ""
    
    for word in sentence.split():
        word = WordCleaner(word)
        cleaned_sentence += word + " "
    
    # 'rstrip()' -> To remove the extra ' ' after the last word
    return cleaned_sentence.rstrip()

In [5]:
# Example use of WordCleaner()

WordCleaner("What's")

'whats'

In [6]:
# Example use of Sentence Cleaner()

SentenceCleaner("Isle of Man TT 2004 Review")

'isle of man tt 2004 review'

In [7]:
# Create a new 'Cleaned Name' column

for i in range(len(data)):
    data.loc[i, 'Cleaned Name'] = SentenceCleaner(data.loc[i, 'Name'])
    
data.head(10)

,Movie_ID,Year,Name,Cleaned Name
0,1,2003,Dinosaur Planet,dinosaur planet
1,2,2004,Isle of Man TT 2004 Review,isle of man tt 2004 review
2,3,1997,Character,character
3,4,1994,Paula Abdul's Get Up & Dance,paula abduls get up dance
4,5,2004,The Rise and Fall of ECW,the rise and fall of ecw
5,6,1997,Sick,sick
6,7,1992,8 Man,8 man
7,8,2004,What the #$*! Do We Know!?,what the do we know
8,9,1991,Class of Nuke 'Em High 2,class of nuke em high 2
9,10,2001,Fighter,fighter


In [8]:
# data = data.drop("Name", axis=1)

In [9]:
# Create a list of unique words in entire dataset


unique_words = set() # Using sets, since set does not store duplicate entries.

# For every movie name
for name in data['Cleaned Name']:
    words = name.split()
    
    # Add every word in the name to the 'unique_words' set
    for word in words:
        unique_words.add(word)

# Convert to list and sort in alphabetical order
unique_words = list(unique_words)
unique_words.sort()

unique_words[-10:]

['zorba',
 'zorro',
 'zoshi',
 'zu',
 'zubeidaa',
 'zulu',
 'zum',
 'zus',
 'zwei',
 'zz']

In [10]:
def TermFrequencies(term, documents):
    '''
    tf(term, document) = count of 'term' in 'document' / number of terms in document
    '''
    
    tmp = []
    
    for document in documents:
        document = document.split()
        tmp.append(document.count(term) / len(document))

    return tmp


def DocumentFrequency(term, documents):
    '''
    df(term) = Occurrence of 'term' in 'N' documents
    '''
    
    count = 0
    
    for document in documents:
        if term in document.split():
            count += 1
            
    return count


def InverseDocumentFrequency(term, documents):
    '''
    idf(term) = log('N' Documents / (df(term) + 1))
    
    Note:
    '1' is to smoothen the value, if the word is absent in the document, since 'df(term)' will be '0'.
    'log' is to dampen the effect of a large corpus (say N >= 10000)
    '''
    
    return math.log(len(documents) / (DocumentFrequency(term, documents) + 1))


def TFIDF(term, documents):
    '''
    tfidf(term, document) = tf(term, document) * idf(term)
    '''
    
    tf = TermFrequencies(term, documents)
    idf = InverseDocumentFrequency(term, documents)
    
    tmp = []
    for idx, document in enumerate(documents):
        tmp.append(tf[idx] * idf)
        
    return tmp

In [11]:
# Cosine Similarity

# cosine(x, y) = x.y / ||x||.||y||
# where ||x|| = sqrt(x1^2 + x2^2 + ... + xn^2)

def CosineSimilarity(x, y):
    tmp = np.dot(x, y)
    
    x_norm = 0
    y_norm = 0
    
    for i in range(len(x)):
        x_norm += x[i] ** 2
        y_norm += x[i] ** 2
        
    x_norm = math.sqrt(x_norm)
    y_norm = math.sqrt(y_norm)
    
    return tmp / (x_norm * y_norm)

In [12]:
# Calculate TFIDF for every movie name, and make a TFIDF table
tfidf = []

for word in unique_words:
    tfidf.append(TFIDF(word, data['Cleaned Name']))

In [13]:
# From (num of unique words x num of documents) -> To (num of documents x num of unique words)

tfidf = np.array(tfidf).T

In [14]:
tfidf.shape

(17770, 12244)

In [21]:
for i in tfidf:
    for j in i:
        if j != 0:
            print(j)
    print()


3.996753725960321
3.0358474272221945

1.0878617305212024
1.3065594620155638
0.7469936688482701
0.3564855331171162
1.3322512419867736
1.5153532900981252

9.092119740588751

1.8184239481177504
1.2002154574460873
1.2768139078973084
1.7373309264961176
1.0935557615224774

0.5311263154223635
1.2033862606145267
1.1082954508699245
0.3564855331171162
1.1910349319222395
0.19543029137887313

7.839356772093383

3.2265312054867463
2.2409810065448106

1.2768139078973084
1.4025356397817834
0.23451634965464777
1.2403495965385174
1.2768139078973084

0.5703731362680386
1.140137990330376
1.2646737239687462
0.9599858717355914
1.447775772080098
0.3564855331171162

6.894895163252532

1.9264563448672152
2.0997431400072015
1.641597774070124
1.7865523978833595

3.0307065801962505
2.337559399636305
1.5618001644414996

0.3388745821237106
0.3677938934771722
0.5755909098580501
0.4040513406150339
0.48611777064222855
0.3388745821237106
0.3290635690311842
0.43524096431566195
0.5303765510194256
0.2705957880630551

4.3


3.573104795766719
4.343327316240294

2.5686084598229533
2.895551544160196
0.39086058275774627

0.825901636350094
0.6649198503117641
0.7199894038016935
0.6081607494150715
0.5856750616655624
0.5951733000378026
0.2673641498378372
0.9991884314900803

2.6131189240311277
2.725276336238199
2.799657520009602

2.6772250611526918
3.6501801356803485

4.343327316240294
2.782879607986295

4.546059870294376
3.0358474272221945

2.895551544160196
1.8270672759815092
0.7129710662342325

3.9196783860466917
3.9196783860466917

1.0018111712727022
0.8820498583577818
0.962963497632182
1.1199081102990547
0.9438875844001073
0.8899884085227546
1.2409506617829411

1.7865523978833595
1.6973836618986764
1.5435872521261182
1.8970105859531194

2.382069863844479
2.664502483973547
0.39086058275774627

0.7940232879481597
0.7940232879481597
0.9332191733365339
0.6860387787227192
0.5472727240874472
0.3551489381794642
0.888167494657849
0.5600371991983829
0.709341059942949

9.092119740588751

1.9598391930233459
1.307847507

1.4774743296700654
0.42778263974053954
0.23451634965464777

2.895551544160196
1.908274636867426
2.664502483973547

7.839356772093383

3.693685824175163
3.4760267885462404

0.6445645654909955
1.4440635127374322
1.131626507220721
0.7513976801835998
1.3681655883964514

9.092119740588751

3.693685824175163
3.9196783860466917

4.546059870294376
2.768385839549669

3.9196783860466917
4.343327316240294

0.5371371379091628
1.5153532900981252
1.1687796998181526
1.15867559618208
1.247113638025775
1.1687796998181526

0.6552528414118387
1.0689545468792359
1.167975572673514
1.2988742486555358
1.2988742486555358
0.6859514713486229
0.1675116783247484

0.6935415691602264
0.8520730144477985
0.9755735480707286
0.5327234072691962
0.529319612996886
0.19543029137887313

2.49422727605155
0.7129710662342325
2.49422727605155

1.2459837719318567
1.131626507220721
0.8963924026179243
0.23451634965464777
1.4154433440092975

2.49422727605155
2.0492269204741036
0.39086058275774627

1.4227305897316491
1.5574797149148

1.1706882576848743
1.3388448935580763
1.8184239481177504
0.23451634965464777

1.7373309264961176
1.2459837719318567
1.5176084687624956
1.5411650758937723
1.4292419183066878

2.799657520009602
0.7129710662342325
2.0492269204741036

2.1887970320934986
0.39086058275774627
2.433453423786899

0.9651838480533986
0.9332191733365339
0.8314090920171834
0.8022575070763511
0.8111511412622996
0.5755663039067339
0.2605737218384975
0.6075309786235985

9.092119740588751

0.8265563400535229
0.8265563400535229
0.39719695626766155
0.29057640396501616
0.8265563400535229
0.3731903920119664
0.8265563400535229
0.6006557355273411
0.8265563400535229
0.6715792407591206
0.10659834075211262

1.058546619443925
0.9927709207766002
1.447775772080098
1.5153532900981252
1.5153532900981252
1.011949142407398

5.160294107864426

9.092119740588751

9.092119740588751

0.583402320256727
0.8398972560028807
0.5231390029548156
0.5231390029548156
0.42778263974053954
0.7993507451920643
0.11725817482732388
0.5170146404307437
0.47


0.9499675293170782
1.2988742486555358
1.167975572673514
1.167975572673514
1.2409506617829411
1.167975572673514
0.5367126287025712

1.3579069295189412
1.6351658017429196
1.5176084687624956
0.46903269930929553

3.0307065801962505
2.2314081559301266
0.39086058275774627

2.895551544160196
2.151020803657831
1.7769732082983964

0.6976315202429524
1.4965363656309303
1.0080669585570894
0.6351835355962634
0.23451634965464777

0.40285285343187216
0.8425930604281593
0.9991884314900803
0.6649198503117641
0.825901636350094
0.53099158176627
0.8846520900058109
0.6147165588366393

3.4760267885462404
3.7413409140773255

0.6373515785068363
1.5678713544186769
1.6351658017429196
0.46903269930929553

3.5386083600232436
4.546059870294376

4.546059870294376
4.343327316240294

0.6373515785068363
1.8184239481177504
1.2295361522844623
0.46903269930929553

8.686654632480588

2.799657520009602
2.5686084598229533
0.39086058275774627

0.49830822874496594
1.0429086101943852
0.8334318860810385
1.0429086101943852
0.6

...


1.1407462725360773
2.5686084598229533
1.058639225993772

0.8057057068637443
1.651803272700188
1.321364312704608
1.3000748606195314

0.31710523647406924
0.31111261978256655
0.8265563400535229
0.49829107526768435
0.41020612387135913
0.8265563400535229
0.6563925057897418
0.3731903920119664
0.8265563400535229
0.10659834075211262
0.7005295799517146

0.8057057068637443
2.0439572521786493
0.5347282996756744
1.9598391930233459

2.895551544160196
1.468456522536079
1.468456522536079

4.546059870294376
3.996753725960321

2.895551544160196
2.151020803657831
1.743796676516052

2.725276336238199
0.39086058275774627
2.1757234610424048

2.273029935147188
1.7102069854955642
2.0439572521786493
0.5347282996756744

3.6501801356803485
3.996753725960321

3.0307065801962505
2.725276336238199
2.725276336238199

0.6844477635216464
0.6373515785068363
1.5678713544186769
0.6351835355962634
1.2459837719318567

0.7966894731335453
2.171663658120147
1.9983768629801606
0.2931454370683097

9.092119740588751

2.3590722

3.3248863526097736
2.478476591923198

3.506339099454458
4.546059870294376

1.0689545468792359
1.199853222861258
0.6111180567721992
0.8673849792063413
0.3350233566494968

2.273029935147188
0.5347282996756744
1.4837798298596596
1.753169549727229

0.7342282612680395
0.5327234072691962
0.954137318433713
0.8552177618318623
1.399828760004801
0.7342282612680395

1.651803272700188
1.9983768629801606
1.06198316353254
2.0997431400072015

2.5950735355070536
3.4760267885462404

0.4281902823558973
0.7635429600026188
0.6802438025595138
0.7635429600026188
0.6218934492711142
0.6636691155782452
0.19444665442751796
0.7896958756800535
0.4709178850146006
0.6802438025595138
0.10659834075211262

0.46040326106499674
1.0314739376695943
1.0553388069071894
0.9849850233217903
0.720047827540778
0.7428999203540179
0.8673849792063413

1.2906124821946987
1.2459837719318567
1.2243410550038103
0.7513976801835998
1.4965363656309303

0.6373515785068363
1.2639062036697943
1.8184239481177504
0.46903269930929553

2.7016201

8.686654632480588

9.092119740588751

0.871898338258026
0.8754445713325734
1.1910349319222395
1.0755104018289154
1.5153532900981252
0.19543029137887313

7.993507451920642

2.2024043636002504
1.0654468145383924
1.5618001644414996

1.062252630844727
2.5686084598229533
1.6659250594555504

3.996753725960321
2.4988875891833255

2.171663658120147
0.5347282996756744
1.8970105859531194
0.2931454370683097

1.9598391930233459
2.0439572521786493
2.273029935147188
0.2931454370683097

2.664502483973547
0.39086058275774627
3.0307065801962505

0.8057057068637443
2.0997431400072015
1.4312059776505694
1.4227305897316491

0.49830822874496594
0.720047827540778
0.8782401087746159
0.45370252542590234
0.1675116783247484
0.6746674125888186
0.8941294852189335

3.3471122338951904
3.6501801356803485

1.7969392554920862
2.4067725212290534
3.0307065801962505

4.087914504357299
3.4760267885462404

3.370372241712637
3.0358474272221945

1.1013423919020593
2.171663658120147
2.171663658120147
1.1013423919020593

1.926

1.8970105859531194
0.2931454370683097

4.546059870294376
0.5862908741366194

1.247113638025775
1.5153532900981252
0.8415114121257001
1.2843042299114766
0.19543029137887313
1.0431510660887557

2.6832131566760493
3.2089855455811116

2.49422727605155
2.1757234610424048
0.39086058275774627

8.175829008714597

3.7413409140773255
4.343327316240294

1.5435872521261182
1.5960173848716355
1.2900735269661066
1.8050793909217902

1.5369201903555778
1.2313636291967562
1.5304263187547626
1.753169549727229

1.2646737239687462
0.8924083537175639
0.8754445713325734
0.3564855331171162
1.2033862606145267
0.19543029137887313

8.686654632480588

1.723723790813133
1.7865523978833595
0.5347282996756744
1.9264563448672152

2.895551544160196
2.216590901739849
3.0307065801962505

1.6797945120057614
0.8953998447494986
1.9991100713466605
1.2403495965385174

2.6263337139977203
4.546059870294376

2.957032955120403
3.0358474272221945

1.0018111712727022
0.9931505110132115
0.8037691196841464
0.8673849792063413
0.6402


1.8184239481177504
1.8184239481177504
1.204813361091027
0.9603320598880721
0.23451634965464777

1.4965363656309303
1.2577518719364436
0.9562641230406476
1.4025356397817834
1.7373309264961176

4.546059870294376
4.546059870294376

4.343327316240294
0.5862908741366194

1.8184239481177504
1.5987014903841286
1.4774743296700654
1.6797945120057614
1.2906124821946987

4.546059870294376
4.546059870294376

3.244715027572184
4.087914504357299

9.092119740588751

0.9562641230406476
1.4600720542721395
1.2295361522844623
0.9370800986648998
1.0262613141982349

1.4440635127374322
0.90245347251699
1.0320588215728852
1.3681655883964514
1.1449647821204556

0.8720394003036904
0.7939794194953291
0.2931454370683097
2.171663658120147

1.399828760004801
1.447775772080098
1.447775772080098
1.399828760004801
1.399828760004801
1.447775772080098

1.8969741196421988
0.7129710662342325
1.7233821347691456

3.4204139709911283
0.5862908741366194

0.855559704402058
0.7939794194953291
2.0439572521786493
0.2931454370683

In [15]:
# This is going to take a LOT of time
# Since it has 17770 x 17770 = 31 Crore Cosine Operations to perform
# At 1 second per operation, this might take 3600+ days to run.

'''
scores = []

for i in range(len(data)):
    tmp = []
    
    for j in range(len(data)):
        tmp.append(CosineSimilarity(tfidf[i], tfidf[j]))
    
    if i % 1000 == 0:
        print(i, "documents done.")
    
    scores.append(tmp)
'''

'\nscores = []\n\nfor i in range(len(data)):\n    tmp = []\n    \n    for j in range(len(data)):\n        tmp.append(CosineSimilarity(tfidf[i], tfidf[j]))\n    \n    if i % 1000 == 0:\n        print(i, "documents done.")\n    \n    scores.append(tmp)\n'

In [16]:
movie_preference = input("Enter a keyword of your favourite movie: ").lower()

# Print the movie names matching the 'movie_preference'
movie_found = False
for idx, movie_name in enumerate(data['Cleaned Name']):
    if movie_preference in movie_name:
        movie_found = True
        print(f"ID: {idx} \t | {data['Name'][idx]}")

if not movie_found:
    print("Movie not found in database, please enter another movie name.")
    
# Get the movie index, so that we can search for similar movies
user_preference_index = int(input("Enter the ID of the movie: "))

Enter a keyword of your favourite movie: jurassic
ID: 4215 	 | Jurassic Park III
ID: 4697 	 | The Lost World: Jurassic Park
ID: 14311 	 | Jurassic Park
Enter the ID of the movie: 4697


In [17]:
def RecommendMovies(movie_index, tfidf, recomm_count=5):
    score = []
    
    # For every movie in the database
    for i in range(len(tfidf)):
        # We don't want to recommend the SAME movie as input
        if i == movie_index: # If the movie to be compared is the input, omit it.
            score.append(0)
        else:
            # Find the cosine similarity between the movies
            score.append(CosineSimilarity(tfidf[movie_index], tfidf[i]))
        
        if i % 1000 == 0:
            print(i, "movies compared.")
    
    # Sort based on movie similarity
    rank = sorted(enumerate(score), key=lambda x: x[1], reverse=True)
    
    # Using the ranked movie indices, get the name of the movie, and return the recommendations
    recommended_movies = []
    for i in range(recomm_count):
        recommended_movies.append(data['Name'][rank[i][0]] + " (" + str(data['Year'][rank[i][0]]) + ")")
        
    return recommended_movies

In [18]:
# Uses cosine similarity score, and gives top 'n' movies
RecommendMovies(user_preference_index, tfidf)

0 movies compared.
1000 movies compared.
2000 movies compared.
3000 movies compared.
4000 movies compared.
5000 movies compared.
6000 movies compared.
7000 movies compared.
8000 movies compared.
9000 movies compared.
10000 movies compared.
11000 movies compared.
12000 movies compared.
13000 movies compared.
14000 movies compared.
15000 movies compared.
16000 movies compared.
17000 movies compared.


['Jurassic Park (1993)',
 'Jurassic Park III (2001)',
 'Mansfield Park (1999)',
 'Echo Park (1986)',
 'MacArthur Park (2003)']

In [19]:
# Experimental CodeBlock:

# Try Recommendation With ONLY TF-IDF,
# using the keyword (movie preference) itself, and not the movie index.

names = sorted(enumerate(TFIDF(movie_preference, data['Cleaned Name'])), reverse=True, key=lambda x: x[1])[:5]

num_movies = 5

print(f"Top {num_movies} movies similar to '{movie_preference}' are:\n")

for i in names:
    print(data['Name'][i[0]] + " (" + str(data['Year'][i[0]]) + ")")
    
# In this method, the distance between movie vectors is considered.
# In the previous method, the angle between the 2 vectors were taken into consideration

Top 5 movies similar to 'jurassic' are:

Jurassic Park (1993)
Jurassic Park III (2001)
The Lost World: Jurassic Park (1997)
Dinosaur Planet (2003)
Isle of Man TT 2004 Review (2004)


<h3> Using Scikit-Learn </h3>

Source: https://medium.com/@sumanadhikari/building-a-movie-recommendation-engine-using-scikit-learn-8dbb11c5aa4b

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

count_matrix = cv.fit_transform(data['Cleaned Name'])

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

tfidf_matrix = tf.fit_transform(data['Cleaned Name'])

In [22]:
print(cv.get_feature_names_out())
print(count_matrix.toarray())

['0014' '0080' '0083' ... 'zus' 'zwei' 'zz']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [23]:
# Calculate Cosine Similarity

from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(tfidf_matrix)
print(similarity_scores)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [24]:
# Let's print the similarity scores for the first word
for idx, i in enumerate(similarity_scores[0]):
    if i > 0:
        print(idx, i)

0 1.0000000000000002
192 0.22666237722315863
587 0.37841850449924125
650 0.3786250275067185
925 0.3139313830426098
1342 0.41397875093916364
1779 0.29948328027607896
2751 0.42435799065299645
4055 0.3740798973890745
4099 0.786089854526672
4699 0.4073126665198927
5252 0.28895609961415736
5935 0.31720036473761243
6046 0.4593776213718236
6938 0.43041571806633855
7301 0.13874973661201387
7701 0.3130418049985273
7958 0.20366714912421394
8531 0.19184192245326917
8663 0.17467033961566072
8666 0.2831362073515337
9159 0.4107555332398031
9186 0.386139002613985
10256 0.23037520593510413
10674 0.41397875093916364
10870 0.37731796223564024
11206 0.20694666704720324
11913 0.17426101079416698
11936 0.2796483687060318
12124 0.21187505287766026
12214 0.29061305931641535
12796 0.2868070197178794
13251 0.17917526554977567
13643 0.2914297582399033
13890 0.3884478737354061
14749 0.2933752403997488
15233 0.46341086609394144
15647 0.3252786186338723
15941 0.27524097351148125
16065 0.3205298792758596
16258 0.26

In [25]:
data['Cleaned Name'][0]

'dinosaur planet'

In [26]:
data['Cleaned Name'][10870]

'fantastic planet'

In [27]:
movie_preference = input("Enter a keyword of your favourite movie: ")

Enter a keyword of your favourite movie: jurassic


In [28]:
movie_found = False

for idx, movie_name in enumerate(data['Cleaned Name']):
    if movie_preference.lower() in movie_name:
        movie_found = True
        print(f"ID: {idx} \t | {data['Name'][idx]}")
        
if not movie_found:
    print("Movie not found in database, please enter another movie name.")

ID: 4215 	 | Jurassic Park III
ID: 4697 	 | The Lost World: Jurassic Park
ID: 14311 	 | Jurassic Park


In [29]:
user_preference_index = int(input("Enter the ID of the movie: "))

Enter the ID of the movie: 4697


In [30]:
similar_movies = list(enumerate(similarity_scores[user_preference_index]))
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)[1:]

In [31]:
sorted_similar_movies

[(14311, 0.802871611838585),
 (4215, 0.6839298382572836),
 (4557, 0.5961519729932231),
 (7045, 0.5961519729932231),
 (13677, 0.5961519729932231),
 (11693, 0.5370259456516571),
 (17018, 0.5370259456516571),
 (2719, 0.4334010346740014),
 (5158, 0.3967172299028006),
 (3455, 0.3486962443418922),
 (12338, 0.325943131959028),
 (3731, 0.3030188438383838),
 (9064, 0.30261619269376955),
 (9235, 0.3023391384445815),
 (10417, 0.3023391384445815),
 (11190, 0.3023391384445815),
 (13662, 0.3023391384445815),
 (15436, 0.3023391384445815),
 (15608, 0.3023391384445815),
 (12290, 0.3000274707438343),
 (14068, 0.2993722078770968),
 (4208, 0.29431807499849993),
 (11843, 0.2909269649337352),
 (11421, 0.2879575541751329),
 (8532, 0.2868963181555023),
 (6339, 0.2863032340334322),
 (16121, 0.2863032340334322),
 (5357, 0.28545676837778233),
 (139, 0.28330294374833626),
 (15518, 0.2816591147586777),
 (842, 0.27639321789986),
 (7050, 0.27639321789986),
 (11657, 0.27639321789986),
 (13766, 0.27639321789986),
 (14

In [32]:
i = 0

print(f"Top 5 movies similar to '{data['Name'][user_preference_index]}' are:\n")

for elt in sorted_similar_movies:
    print(f"#{i+1} {data['Name'][elt[0]]} ({data['Year'][elt[0]]})")
    i += 1
    
    if i == 5: break

Top 5 movies similar to 'The Lost World: Jurassic Park' are:

#1 Jurassic Park (1993)
#2 Jurassic Park III (2001)
#3 The Lost World (2001)
#4 The Lost World (1992)
#5 The Lost World (1925)


<h2>Collaborative Filtering based Recommendation</h2>

Inspiration: https://www.youtube.com/watch?v=3oCtj29XeYY

In [33]:
rows = len(data)
rows

17770

In [34]:
def AddUser(pref, non_pref):
    pref_movies = []
    non_pref_movies = []

    # Get list of preferred movies & non-preferred movies
    for i in range(rows):
        if pref in data['Cleaned Name'][i]:
            pref_movies.append(i)
        if non_pref in data['Cleaned Name'][i]:
            non_pref_movies.append(i)
            
    user = []
    
    good_ratings = [8, 9, 10]
    bad_ratings = [2, 3, 4]
    watched = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1] # Giving more '1's, so there is a '1' in '10' probability movie is NOT watched.
    
    for i in range(rows):               
        if i in pref_movies:
            # Give a random chance that they watched a movie
            # They need not have watched every movie in given genre
            if random.choice(watched):
                user.append(random.choice(good_ratings))
            else:
                user.append(np.nan)
        elif i in non_pref_movies:
            if random.choice(watched):
                user.append(random.choice(bad_ratings))
            else:
                user.append(np.nan)
        else: # User did not interact with movie
            user.append(np.nan)

    user = pd.DataFrame(user).T
    user.columns = data['Cleaned Name']
    return engagement.append(user)

In [35]:
# Let's create an Engagement Table
engagement = pd.DataFrame(columns=data['Cleaned Name'])

# Let's create a User's profile who likes Dogs, and doesn't like Dance movies
engagement = AddUser(pref='dog', non_pref='dance')

# Let's create a User's profile who likes fighting, and doesn't like Romancee
engagement = AddUser(pref='fight', non_pref='romance')

# Let's create a User's profile who likes violent movies, and doesn't like Child movies
engagement = AddUser(pref='kill', non_pref='child')

# Let's create a User's profile who likes Dog movies, and doesn't like Child movies
engagement = AddUser(pref='dog', non_pref='child')

engagement = engagement.reset_index(drop=True)

/var/folders/67/4mxjfv5j26z772wt9gjhy6gw0000gn/T/ipykernel_39970/160088181.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return engagement.append(user)
/var/folders/67/4mxjfv5j26z772wt9gjhy6gw0000gn/T/ipykernel_39970/160088181.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return engagement.append(user)
/var/folders/67/4mxjfv5j26z772wt9gjhy6gw0000gn/T/ipykernel_39970/160088181.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return engagement.append(user)
/var/folders/67/4mxjfv5j26z772wt9gjhy6gw0000gn/T/ipykernel_39970/160088181.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return engagement.append(user)


In [36]:
# To normalize the score from '0' to '10' -> To make it [-5, 5]
engagement = engagement.apply(lambda x: x - 5)

In [37]:
engagement.head(10)


Cleaned Name,dinosaur planet,isle of man tt 2004 review,character,paula abduls get up dance,the rise and fall of ecw,sick,8 man,what the do we know,class of nuke em high 2,fighter,...,levity,gattaca,interiors,shakespeare in love,godzillas revenge,where the wild things are and other maurice sendak stories,fidel castro american experience,epoch,the company,alien hunter
0,NaN,NaN,NaN,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Consider '0' score as 'NOT Interacted'
for i in range(len(engagement)):
    for j in range(len(engagement.T)):
        if np.isnan(engagement.iloc[i, j]):
            engagement.iloc[i, j] = 0

In [39]:
engagement = engagement.T

In [40]:
engagement.head()

,0,1,2,3
Cleaned Name,,,,
dinosaur planet,0.0,0.0,0.0,0.0
isle of man tt 2004 review,0.0,0.0,0.0,0.0
character,0.0,0.0,0.0,0.0
paula abduls get up dance,-3.0,0.0,0.0,0.0
the rise and fall of ecw,0.0,0.0,0.0,0.0


In [41]:
# To see the movie names with 'dog' in it

dog_movies = []
for i in data['Cleaned Name']:
    if 'dog' in i:
        dog_movies.append(i)
        
dog_movies[:15]

['where sleeping dogs lie',
 'reservoir dogs',
 'road dogz',
 'ghost dog the way of the samurai',
 'dogma',
 'lawn dogs',
 'a dog of flanders',
 'underdog chronicles',
 'the plague dogs',
 'lie down with dogs',
 'man bites dog',
 'reservoir dogs bonus material',
 'black dog',
 'charles mingus triumph of the underdog',
 'hot dog the movie']

In [42]:
# Now, let's find the prediction for User 4
new_user = 3

# Find similarity of a user with others
similarity_matrix = []

for i in range(len(engagement.T)):
    # Obviously, User 4 is MOST similar to User 4, so make the score '0' intentionally
    if i == new_user:
        similarity_matrix.append(0)
    else:
        similarity_matrix.append(CosineSimilarity(engagement[new_user], engagement[i]))

In [43]:
similarity_matrix

[0.6870397643593521, 0.00662739322533137, 0.1597938144329897, 0]

In [44]:
sorted(enumerate(similarity_matrix), key=lambda x: x[1], reverse=True)

[(0, 0.6870397643593521),
 (2, 0.1597938144329897),
 (1, 0.00662739322533137),
 (3, 0)]

In [45]:
most_similar_user = sorted(enumerate(similarity_matrix), key=lambda x: x[1], reverse=True)[0][0]

In [46]:
print(f"Most similar user is: User {most_similar_user + 1}")

Most similar user is: User 1


In [47]:
engagement

,0,1,2,3
Cleaned Name,,,,
dinosaur planet,0.0,0.0,0.0,0.0
isle of man tt 2004 review,0.0,0.0,0.0,0.0
character,0.0,0.0,0.0,0.0
paula abduls get up dance,-3.0,0.0,0.0,0.0
the rise and fall of ecw,0.0,0.0,0.0,0.0
...,...,...,...,...
where the wild things are and other maurice sendak stories,0.0,0.0,0.0,0.0
fidel castro american experience,0.0,0.0,0.0,0.0
epoch,0.0,0.0,0.0,0.0


In [48]:
n_movies = 5

print(f"Top {n_movies} recommended movies based on similar Users are: \n")

i = 0
for movie in sorted(enumerate(list(engagement.iloc[:, most_similar_user])), key=lambda x: x[1], reverse=True):
    if engagement[new_user][movie[0]] == 0:
        print(f"#{i+1} {data['Name'][movie[0]]} ({data['Year'][movie[0]]})")
        i += 1
        
    if i >= n_movies: break

Top 5 recommended movies based on similar Users are: 

#1 The Plague Dogs (1982)
#2 The Corndog Man (1999)
#3 Dog Gone (2003)
#4 How to Kill Your Neighbor's Dog (2001)
#5 Dodgeball: A True Underdog Story (2004)


In [49]:
# To Experiment:
# 1. With more data pre-processing (i.e.) Stemming, Lemmatization, & more
# 2. Hybrid Recommendation System = Collaborative-filtering + Content-based
# 3. Put code it into more functions, for ease-of-use